# Simulation Experiments: Computational Validation of GENERIC Structure

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lawrennd/tig-code/blob/main/examples/simulation_experiments_inaccessible_game.ipynb)

## Neil D. Lawrence

### October 2025

This notebook generates all computational results and figures for simulation studies in "The Inaccessible Game" paper.

*System:* Three binary variables with pairwise interactions (Ising model / Boltzmann machine)

*Goal:* Validate theoretical predictions:

- GENERIC decomposition $M = S + A$ emerges from constraint geometry
- Marginal entropy conservation ($\sum_i h_i = C$) is maintained during evolution
- Constrained vs unconstrained dynamics behave differently
- Regime variation $\tfrac{\|A\|}{\|S\|}$ depends on local geometry


In [ ]:
# Setup cell for Google Colab compatibility
import sys
import os

# Check if we're running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running in Google Colab - installing TIG package...")
    
    # Install the tig package directly from GitHub
    %pip install -q git+https://github.com/lawrennd/tig-code.git
    
    print("✓ TIG package installed!")
    print("✓ Setup complete! You can now run the notebook.")
else:
    print("💻 Running locally")
    print("   Make sure you have installed: pip install -e .")


# Initial Setup


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Set plot style
plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')
big_wide_figsize = (10, 5)
big_figsize = (8, 8)
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 'medium',
})

In [ ]:
verbose = False
N = 3  # Three binary variables

print(f"System: {N} binary variables with pairwise interactions")
print(f"Parameter space dimension: d = {N + N*(N-1)//2} (marginals + interactions)")

In [ ]:
import numpy as np
from tig import generic_decomposition_n3 as gd

Random initial parameters (for initial demonstration)


In [ ]:
h = 0 # Set external field
theta_frustrated = np.array([h, h, h, 1, -1, 1]) # Set frustrated parameters
theta_frustrated = theta_frustrated/np.sqrt(theta_frustrated.T@theta_frustrated) # normalise

In [ ]:
print(f"  θ₁, θ₂, θ₃ = {theta_frustrated[:3]}")
print(f"  θ₁₂, θ₁₃, θ₂₃ = {theta_frustrated[3:]}")

In [ ]:
# Compute initial constraint value
marginals, _ = gd.compute_marginals(theta_frustrated, N)
C_init = sum(gd.marginal_entropy(m) for m in marginals)

In [ ]:
print(f"\nInitial marginal entropies:")
for i, m in enumerate(marginals, 1):
    print(f"  h_{i} = {gd.marginal_entropy(m):.4f}")
print(f"\nConstraint value: C = Σh_i = {C_init:.6f}")
print(f"Maximum possible: {N*np.log(2):.6f} (uniform distribution)")

In [ ]:
result = gd.analyse_generic_structure(theta_frustrated, N)

In [ ]:
print(f"  Jacobian M = S + A")
print(f"  ||S|| = {result['norm_S']:.3f} (dissipative, symmetric)")
print(f"  ||A|| = {result['norm_A']:.3f} (conservative, antisymmetric)")
print(f"  ||A||/||S|| = {result['ratio']:.3f}")

# Vary Temperature

In this experiment we scale the parameters to change the temperature of the system and monitor the ratio between the antisymmetric and symmetric parts.

In [ ]:
theta = theta_frustrated
temp_values = np.logspace(-1.5, 1.5, 200) 
beta_values = 1/temp_values
ratios_temp = []
norms_S_temp = []
norms_A_temp = []

print("Computing ||A||/||S|| for β ∈ [0.1, 3.0]...")
for i, beta in enumerate(beta_values):
    theta_scaled = beta * theta
    try:
        result_temp = gd.analyse_generic_structure(theta_scaled, N)
        ratios_temp.append(result_temp['ratio'])
        norms_S_temp.append(result_temp['norm_S'])
        norms_A_temp.append(result_temp['norm_A'])
        if (i+1) % 5 == 0 and verbose:
            print(f"  β = {beta:.2f}: ||A||/||S|| = {result_temp['ratio']:.3f}, ||S|| = {result_temp['norm_S']:.3f}, ||A|| = {result_temp['norm_A']:.3f}")
    except:
        ratios_temp.append(np.nan)
        norms_S_temp.append(np.nan)
        norms_A_temp.append(np.nan)
        print(f"  β = {beta:.2f}: Failed")

ratios_temp = np.array(ratios_temp)
norms_S_temp = np.array(norms_S_temp)
norms_A_temp = np.array(norms_A_temp)

# Find peak
valid_mask = ~np.isnan(ratios_temp)
peak_idx = np.argmax(ratios_temp[valid_mask])
peak_beta = beta_values[valid_mask][peak_idx]
peak_ratio = ratios_temp[valid_mask][peak_idx]

print(f"  Peak ratio: ||A||/||S|| = {peak_ratio:.3f}")
print(f"  Peak location: β = {peak_beta:.2f}")
print(f"  Low temperature (β={beta_values[0]:.3f}): ||A||/||S|| = {ratios_temp[0]:.3f}")
print(f"  High temperature (β={beta_values[-1]:.3f}): ||A||/||S|| = {ratios_temp[-1]:.3f}")


Plot the Frobenius norms of the antisymmetric and symmetric components.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# generate component norms separately vs temperature
fig, ax = plt.subplots(figsize=gd.big_wide_figsize)
ax.semilogx(temp_values, norms_S_temp[valid_mask], 'r--', linewidth=4, 
        label=r'$\|S\|$ (dissipative)')
ax.semilogx(temp_values, norms_A_temp[valid_mask], 'b-', linewidth=4, 
        label=r'$\|A\|$ (conservative)')
ax.set_xlabel(r'inverse temperature $\beta$')
ax.set_ylabel('Frobenius Norm')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.savefig('./diagrams/fig-n3_coldness-vs-component_norms.pdf', bbox_inches='tight', dpi=300)

Plot the ratio of the Frobenius norms vs inverse temperature ($\beta$)

In [ ]:
# Plot ratio vs β 
beta_values = 1.0 / temp_values[valid_mask]  # β^-1 ∝ T

fig, ax = plt.subplots(figsize=gd.big_wide_figsize)
ax.semilogx(temp_values, ratios_temp[valid_mask], 'b-', linewidth=4)
peak_temp = 1.0 / peak_beta
ax.axvline(peak_temp, color='r', linestyle='--', linewidth=2, alpha=0.5)
ax.set_xlabel(r'inverse temperature $\beta$')
ax.set_ylabel(r'$\|A\|/\|S\|$')
ax.grid(True, alpha=0.3)
ax.text(peak_temp, peak_ratio*1.1, f'Peak: β={peak_temp:.2f}', 
        ha='center', fontsize=16)

plt.savefig('./diagrams/fig-n3_coldness-vs-ratio.pdf', bbox_inches='tight', dpi=300)

# Experiment 2: Constraint Maintenance During Evolution

Compare constrained dynamics (with $\sum_i h_i = C$) to unconstrained
maximum entropy production (pure gradient ascent on $H$).



In [ ]:
sol_constrained = gd.solve_constrained_maxent(
    theta_frustrated, N=3, n_steps=40000, dt=0.01, verbose=verbose
)

Summary of the constrained dynamics.

In [ ]:
print(f"  Converged: {sol_constrained['converged']}")
print(f"  Steps: {sol_constrained['n_steps']}")
print(f"  Final ||F(θ)||: {sol_constrained['flow_norms'][-1]:.2e}")
print(f"  Final constraint violation: |ΔC| = {abs(sol_constrained['constraint_values'][-1] - C_init):.2e}")
print(f"  Initial sum of marginals = {sol_constrained['constraint_values'][0]}")
print(f"  Initial sum of marginals = {sol_constrained['constraint_values'][-1]}")


# Experiment 3: Constrained vs Unconstrained Comparison

Integrating unconstrained dynamics from same initial condition...

(Pure maximum entropy production without constraint)

**Trajectory comparison**

-   Both start from same initial condition

-   Unconstrained converges to $\boldsymbol{\theta} = \mathbf{0}$
    (maximum entropy state)

-   Constrained converges to different point on manifold
    $\{\boldsymbol{\theta}: \sum_i h_i = C\}$

-   Demonstrates how constraint shapes parameter space flow

**Entropy evolution**

-   Both increase joint entropy $H(X_1,X_2,X_3)$ (second law holds)

-   Only constrained preserves $\sum_i h_i = C$ exactly

-   Unconstrained allows $\sum_i h_i \to \text{maximum}$ (independent
    distribution)

-   Different rates of entropy production due to constraint



In [ ]:
sol_unconstrained = gd.solve_unconstrained_maxent(
    theta_frustrated, N=3, n_steps=40000, dt=0.01, verbose=verbose
)

In [ ]:
print(f"\nUnconstrained Dynamics Summary:")
print(f"  Converged: {sol_unconstrained['converged']}")
print(f"  Steps: {sol_unconstrained['n_steps']}")
print(f"  Final ||F(θ)||: {sol_unconstrained['flow_norms'][-1]:.2e}")
print(f"  Final ||θ||: {np.linalg.norm(sol_unconstrained['trajectory'][-1]):.3f}")

Compare final states between constrated and unconstrained.

In [ ]:
theta_final_c = sol_constrained['trajectory'][-1]
theta_final_u = sol_unconstrained['trajectory'][-1]

print(f"  Constrained:   ||θ|| = {np.linalg.norm(theta_final_c):.3f}")
print(f"  Unconstrained: ||θ|| = {np.linalg.norm(theta_final_u):.3f}")


In [ ]:
# Compute final entropies
marginals_c, _ = gd.compute_marginals(theta_final_c, N)
marginals_u, _ = gd.compute_marginals(theta_final_u, N)


Compare final marginal entropy sums.

In [ ]:
C_final_c = sum(gd.marginal_entropy(m) for m in marginals_c)
C_final_u = sum(gd.marginal_entropy(m) for m in marginals_u)

print(f"\nFinal marginal entropy sums:")
print(f"  Constrained:   Σh_i = {C_final_c:.6f} (= C_init)")
print(f"  Unconstrained: Σh_i = {C_final_u:.6f} (≠ C_init)")

In [ ]:
traj_c = sol_constrained['trajectory']
traj_u = sol_unconstrained['trajectory']

In [ ]:
# %load -s save_trajectory_comparison /Users/neil/lawrennd/the-inaccessible-game/generic_decomposition_n3.py
fig, ax = plt.subplots(figsize=gd.big_figsize)

# Plot trajectories
ax.plot(traj_u[:, 0], traj_u[:, 1], 'r--', alpha=0.6, linewidth=4, 
        label='Unconstrained')
ax.plot(traj_c[:, 0], traj_c[:, 1], 'b-', alpha=0.7, linewidth=4,
        label='Constrained')

# Mark initial and final points
ax.plot(traj_c[0, 0], traj_c[0, 1], 'go', markersize=12, 
        label='Initial', zorder=5)
ax.plot(traj_u[-1, 0], traj_u[-1, 1], 'r*', markersize=16, zorder=5)
ax.plot(traj_c[-1, 0], traj_c[-1, 1], 'bs', markersize=12, zorder=5)

ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)
ax.axvline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_trajectory-comparison.pdf', bbox_inches='tight', dpi=300)


In [ ]:
# %load -s save_trajectory_comparison /Users/neil/lawrennd/the-inaccessible-game/generic_decomposition_n3.py
fig, ax = plt.subplots(figsize=gd.big_figsize)

# Plot trajectories
ax.plot(traj_u[:, 3], traj_u[:, 4], 'r--', alpha=0.6, linewidth=4, 
        label='Unconstrained')
ax.plot(traj_c[:, 3], traj_c[:, 4], 'b-', alpha=0.7, linewidth=4,
        label='Constrained')

# Mark initial and final points
ax.plot(traj_c[0, 3], traj_c[0, 4], 'go', markersize=12, 
        label='Initial', zorder=5)
ax.plot(traj_u[-1, 3], traj_u[-1, 4], 'r*', markersize=16, zorder=5)
ax.plot(traj_c[-1, 3], traj_c[-1, 4], 'bs', markersize=12, zorder=5)

ax.set_xlabel(r'$\theta_{12}$')
ax.set_ylabel(r'$\theta_{13}$')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)
ax.axvline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_trajectory-interaction-comparison.pdf', bbox_inches='tight', dpi=300)

In [ ]:
# %load -s save_marginal_entropy_evolution /Users/neil/lawrennd/the-inaccessible-game/generic_decomposition_n3.py
fig, ax = plt.subplots(figsize=gd.big_wide_figsize)

# Compute marginal entropy sum for both
steps_u, sum_h_u = gd.compute_marginal_entropy_trajectory(sol_unconstrained['trajectory'], N, sample_every=1)
steps_c, sum_h_c = gd.compute_marginal_entropy_trajectory(sol_constrained['trajectory'], N, sample_every=1)

ax.plot(steps_u, sum_h_u, 'r--', linewidth=4, label='Unconstrained')
ax.plot(steps_c, sum_h_c, 'b-', linewidth=4, label='Constrained')

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$\sum_i h_i$')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_marginal-entropy-evolution.pdf', bbox_inches='tight', dpi=300)


In [ ]:
# %load -s save_joint_entropy_evolution /Users/neil/lawrennd/the-inaccessible-game/generic_decomposition_n3.py

fig, ax = plt.subplots(figsize=gd.big_wide_figsize)

# Compute joint entropy for both trajectories
steps_u, H_u = gd.compute_joint_entropy_trajectory(sol_unconstrained['trajectory'], N, sample_every=1)
steps_c, H_c = gd.compute_joint_entropy_trajectory(sol_constrained['trajectory'], N, sample_every=1)

ax.plot(steps_u, H_u, 'r--', linewidth=4, label='Unconstrained')
ax.plot(steps_c, H_c, 'b-', linewidth=4, label='Constrained')
ax.axhline(N*np.log(2), color='k', linestyle='--', linewidth=2, 
           alpha=0.5, label='Maximum')

ax.set_xlabel(r'$\tau$')
ax.set_ylabel(r'$H(X_1,X_2,X_3)$')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_joint-entropy-evolution.pdf', bbox_inches='tight', dpi=300)


# Plot External Field Parameters

Plot of $\theta_1$ against $\theta_3$. The behaviour of $\theta_3$ is identical to $\theta_1$. 

In [ ]:
fig, ax = plt.subplots(figsize=gd.big_wide_figsize)

traj_u = sol_unconstrained['trajectory']
traj_c = sol_constrained['trajectory']

steps_u = np.arange(len(traj_u))
steps_c = np.arange(len(traj_c))

labels = [r'$\theta_1$', r'$\theta_2$', r'$\theta_3$']
colors = ['C0', 'C1', 'C2']

for i, (label, color) in enumerate(zip(labels, colors)):
    if i == 2: # don't plot theta_3
        continue
    ax.plot(steps_u, traj_u[:, i], '--', alpha=0.5, linewidth=3, 
            color=color, label=f'{label} (unc.)')
    ax.plot(steps_c, traj_c[:, i], '-', linewidth=4, 
            color=color, label=f'{label} (con.)')

ax.set_xlabel(r'$\tau$')
ax.set_ylabel('parameter value')
ax.legend(fontsize=14, ncol=2)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_marginal-parameters-evolution.pdf', bbox_inches='tight', dpi=300)


# Plot Interaction Parameters


Plot of $\theta_{12}$ against $\theta_{13}$. The behaviour of $\theta_{23}$ is identical to $\theta_{12}$. 

In [ ]:
fig, ax = plt.subplots(figsize=gd.big_wide_figsize)
    
traj_u = sol_unconstrained['trajectory']
traj_c = sol_constrained['trajectory']

steps_u = np.arange(len(traj_u))
steps_c = np.arange(len(traj_c))

labels = [r'$\theta_{12}$', r'$\theta_{13}$', r'$\theta_{23}$']
colors = ['C0', 'C1', 'C2']

for i, (label, color) in enumerate(zip(labels, colors)):
    if i == 2:
        continue
    param_idx = 3 + i  # Interaction parameters start at index 3
    ax.plot(steps_u, traj_u[:, param_idx], '--', alpha=0.5, linewidth=3,
            color=color, label=f'{label} (unc.)')
    ax.plot(steps_c, traj_c[:, param_idx], '-', linewidth=4,
            color=color, label=f'{label} (con.)')

ax.set_xlabel('$\tau$')
ax.set_ylabel('parameter value')
ax.legend(fontsize=14, ncol=2)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=1, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_interaction-parameters-evolution.pdf', bbox_inches='tight', dpi=300)


In [4]:
import numpy as np
from tig import generic_decomposition_n3 as gd


theta1 = np.array([0.5, -0.3, 0.2, -0.8, 0.7, -0.1])
result1 = gd.verify_jacobi_identity(theta1, N=3, eps_diff=1e-5, verbose=True)

theta2 = np.array([0.1, 0.1, 0.1, 0.0, 0.0, 0.0])
result2 = gd.verify_jacobi_identity(theta2, N=3, eps_diff=1e-5, verbose=True)

theta3 = np.zeros(6)
result3 = gd.verify_jacobi_identity(theta3, N=3, eps_diff=1e-5, verbose=True)

for i in range(10):
    theta4 = np.random.randn(6)
    result4 = gd.verify_jacobi_identity(theta4, N=3, eps_diff=1e-5, verbose=True)



JACOBI IDENTITY VERIFICATION
Parameter dimension: d = 6
Number of triplets tested: 216

Results:
  Maximum violation:  8.026706e-05
  Mean violation:     2.346962e-05
  ||A|| (Frobenius):  0.013962
  Relative violation: 4.117825e-01 (scaled by ||A||²)

Worst triplet: (θ_3, θ_0, θ_5)
  {{θ_3, θ_0}, θ_5} = 8.026706e-05

✓ Jacobi identity satisfied (violation < 1e-4)


JACOBI IDENTITY VERIFICATION
Parameter dimension: d = 6
Number of triplets tested: 216

Results:
  Maximum violation:  3.508126e-08
  Mean violation:     6.985783e-09
  ||A|| (Frobenius):  0.021961
  Relative violation: 7.273813e-05 (scaled by ||A||²)

Worst triplet: (θ_1, θ_2, θ_4)
  {{θ_1, θ_2}, θ_4} = 3.508126e-08

✓ Jacobi identity satisfied (violation < 1e-4)


JACOBI IDENTITY VERIFICATION
Parameter dimension: d = 6
Number of triplets tested: 216

Results:
  Maximum violation:  1.394825e+01
  Mean violation:     3.985973e+00
  ||A|| (Frobenius):  0.030930
  Relative violation: 1.458054e+04 (scaled by ||A||²)

Worst tr

In [5]:
"""
Test Jacobi identity for N=3 binary variables numerically.

Compare with harmonic oscillator findings to see if geometric pattern holds.
"""

import numpy as np
from tig import generic_decomposition_n3 as gd

print("="*70)
print("NUMERICAL JACOBI IDENTITY TESTS FOR N=3 BINARY VARIABLES")
print("="*70)
print()
print("Testing to see if Jacobi violations show similar geometric patterns")
print("as the harmonic oscillator (6/27 violations for asymmetric cases).")
print()
print("Parameter ordering: [θ_1, θ_2, θ_3, θ_12, θ_13, θ_23]")
print("                     [marginals ↑    | interactions ↑  ]")
print()

# Test 1: Symmetric point (all marginals equal, all interactions equal)
print("="*70)
print("TEST 1: Fully symmetric (all θ_i equal, all θ_ij equal)")
print("="*70)
theta1 = np.array([0.1, 0.1, 0.1, 0.0, 0.0, 0.0])
result1 = gd.verify_jacobi_identity(theta1, N=3, eps_diff=1e-6, verbose=True)

# Test 2: Asymmetric marginals, no interactions
print("\n" + "="*70)
print("TEST 2: Asymmetric marginals, no interactions")
print("="*70)
theta2 = np.array([0.5, -0.3, 0.2, 0.0, 0.0, 0.0])
result2 = gd.verify_jacobi_identity(theta2, N=3, eps_diff=1e-6, verbose=True)

# Test 3: Symmetric marginals, asymmetric interactions
print("\n" + "="*70)
print("TEST 3: Symmetric marginals, asymmetric interactions")
print("="*70)
theta3 = np.array([0.1, 0.1, 0.1, -0.8, 0.7, -0.1])
result3 = gd.verify_jacobi_identity(theta3, N=3, eps_diff=1e-6, verbose=True)

# Test 4: Frustrated system (from paper)
print("\n" + "="*70)
print("TEST 4: Frustrated system (paper example)")
print("="*70)
theta4 = np.array([0.5, -0.3, 0.2, -0.8, 0.7, -0.1])
result4 = gd.verify_jacobi_identity(theta4, N=3, eps_diff=1e-6, verbose=True)

# Test 5: Uniform marginals, weak interactions
print("\n" + "="*70)
print("TEST 5: Uniform marginals, weak uniform interactions")
print("="*70)
theta5 = np.array([0.0, 0.0, 0.0, 0.1, 0.1, 0.1])
result5 = gd.verify_jacobi_identity(theta5, N=3, eps_diff=1e-6, verbose=True)

# Summary
print("\n" + "="*70)
print("SUMMARY OF NUMERICAL TESTS")
print("="*70)
print(f"Test 1 (fully symmetric):     max violation = {result1['max_violation']:.6e}")
print(f"Test 2 (asym marginals):      max violation = {result2['max_violation']:.6e}")
print(f"Test 3 (asym interactions):   max violation = {result3['max_violation']:.6e}")
print(f"Test 4 (frustrated):          max violation = {result4['max_violation']:.6e}")
print(f"Test 5 (uniform+weak int):    max violation = {result5['max_violation']:.6e}")
print("="*70)
print()

# Analyze violation patterns
print("="*70)
print("ANALYZING VIOLATION PATTERNS")
print("="*70)
print()

threshold = 1e-6

for test_name, result in [
    ("Fully symmetric", result1),
    ("Asym marginals", result2),
    ("Asym interactions", result3),
    ("Frustrated", result4),
    ("Uniform+weak int", result5)
]:
    violations = result['violations']
    nonzero_indices = np.where(np.abs(violations) > threshold)
    num_nonzero = len(nonzero_indices[0])
    total_triplets = result['num_triplets']
    
    print(f"{test_name:20s}: {num_nonzero:3d}/{total_triplets} triplets violate (>{threshold:.0e})")
    print(f"                       Max: {result['max_violation']:.3e}, Mean: {result['mean_violation']:.3e}")
    
    if num_nonzero > 0 and num_nonzero <= 20:
        print("                       Violating triplets:")
        for idx in range(num_nonzero):
            i, j, k = nonzero_indices[0][idx], nonzero_indices[1][idx], nonzero_indices[2][idx]
            val = violations[i, j, k]
            print(f"                         ({i},{j},{k}): {val:.6e}")
    print()

print("="*70)
print("COMPARISON WITH HARMONIC OSCILLATOR")
print("="*70)
print()
print("Harmonic oscillator findings:")
print("  - Symmetric case (θ_xx = θ_pp): All 27 triplets satisfy")
print("  - Asymmetric case: Exactly 6/27 triplets violate")
print("  - Pattern: violations in mixed-index triplets")
print()
print("N=3 binary system:")
total = result1['num_triplets']
print(f"  - Fully symmetric: {np.sum(np.abs(result1['violations']) > threshold)}/{total} violations")
print(f"  - Asymmetric marginals: {np.sum(np.abs(result2['violations']) > threshold)}/{total} violations")
print(f"  - Asymmetric interactions: {np.sum(np.abs(result3['violations']) > threshold)}/{total} violations")
print(f"  - Frustrated: {np.sum(np.abs(result4['violations']) > threshold)}/{total} violations")
print()

if result1['max_violation'] < 1e-6:
    print("✓ Pattern MATCHES: Symmetric case satisfies Jacobi identity")
else:
    print("✗ Pattern DIFFERS: Even symmetric case shows violations")

print()
print("="*70)



NUMERICAL JACOBI IDENTITY TESTS FOR N=3 BINARY VARIABLES

Testing to see if Jacobi violations show similar geometric patterns
as the harmonic oscillator (6/27 violations for asymmetric cases).

Parameter ordering: [θ_1, θ_2, θ_3, θ_12, θ_13, θ_23]
                     [marginals ↑    | interactions ↑  ]

TEST 1: Fully symmetric (all θ_i equal, all θ_ij equal)

JACOBI IDENTITY VERIFICATION
Parameter dimension: d = 6
Number of triplets tested: 216

Results:
  Maximum violation:  8.717381e-07
  Mean violation:     1.304539e-07
  ||A|| (Frobenius):  0.021961
  Relative violation: 1.807537e-03 (scaled by ||A||²)

Worst triplet: (θ_1, θ_2, θ_3)
  {{θ_1, θ_2}, θ_3} = -8.717381e-07

✓ Jacobi identity satisfied (violation < 1e-4)


TEST 2: Asymmetric marginals, no interactions

JACOBI IDENTITY VERIFICATION
Parameter dimension: d = 6
Number of triplets tested: 216

Results:
  Maximum violation:  7.280321e-05
  Mean violation:     9.291492e-06
  ||A|| (Frobenius):  0.018967
  Relative violation: 